## Tryout functions

In [6]:
import os
import importlib
import helperfunctions
from VisualisationTools import soccer_animation

# Clear cache and reload the helperfunctions module
importlib.reload(helperfunctions)
importlib.reload(soccer_animation)

# Import the required functions after reloading
from helperfunctions import get_database_connection, fetch_team_matches, calculate_ball_possession


In [2]:
conn = get_database_connection()
team_name = "anderlecht"
games = fetch_team_matches(team_name, conn)

# Display the team_id of the given team_name
team_id = games.loc[games['home_team_name'].str.lower() == team_name.lower(), 'home_team_id'].iloc[0]
print(f"Team ID for {team_name}: {team_id}")
conn.close()


OperationalError: connection to server at "fuji.ucll.be" (193.190.58.68), port 62425 failed: FATAL:  server login has been failing, cached error: database "local_r0887380" does not exist (server_login_retry)


In [ ]:
print(games)

In [ ]:
# Re-establish the database connection
conn = get_database_connection()

# Fetch match events for the first game in the games dataframe
match_id = games.iloc[0]['match_id']
match_events = helperfunctions.fetch_match_events(match_id, conn)

# Close the connection after fetching the data
conn.close()

print(match_events.head(10))
print(match_events.describe())
print(match_events.info())

In [ ]:
import pandas as pd

# Re-establish the database connection
conn = get_database_connection()

# Fetch match events for the first game in the games dataframe
match_id = games.iloc[0]['match_id']
changes = calculate_ball_possession(match_id, conn,team_id=team_id)

# Close the connection after fetching the data
conn.close()

print(changes.head(10))


In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt

# Filter the data to include only rows where time_difference_seconds is between -500 and 500
changes['time_difference_seconds'] = changes['time_difference'].dt.total_seconds()

# Print negative time_difference_seconds values
negative_time_diff = changes[changes['time_difference_seconds'] < 0]
print("Negative time_difference_seconds values:")
print(negative_time_diff[['time_difference_seconds']])

# Set negative time_difference_seconds to 0
changes.loc[changes['time_difference_seconds'] < 0, 'time_difference_seconds'] = 0

# Create a histogram with facets for ball_possession and a bin size of 5 seconds
sns.histplot(data=changes, x='time_difference_seconds', hue='ball_possession', multiple='dodge', bins=range(-500, 505, 5))

# Add labels and title
plt.xlabel('Time Difference (seconds)')
plt.ylabel('Frequency')
plt.title('Histogram of Ball Possession Changes for Anderlecht')
plt.legend(title='Ball Possession', labels=['Opponent', 'Anderlecht'])

# Show the plot
plt.show()


In [ ]:
# Filter the data to include only rows where time_difference_seconds is between -60 and 60
filtered_changes_60 = changes[(changes['time_difference_seconds'] >= -60) & (changes['time_difference_seconds'] <= 60)]

# Create a histogram with facets for ball_possession and a bin size of 5 seconds
sns.histplot(data=filtered_changes_60, x='time_difference_seconds', hue='ball_possession', multiple='dodge', bins=range(0, 65, 5))

# Add labels and title
plt.xlabel('Time Difference (seconds)')
plt.ylabel('Frequency')
plt.title('Histogram of Ball Possession Changes for Anderlecht (Zoomed In)')
plt.legend(title='Ball Possession', labels=['Opponent', 'Anderlecht'])

# Show the plot
plt.show()

# Animation between two timestamps

! Create a  .env file with the following content:
 ```env
PG_HOST=fuji.ucll.be
PG_PORT=52425
PG_USER=your_user
PG_PASSWORD=your_password   

```

In [7]:
import dotenv

dotenv.load_dotenv()
import os


# Load environment variables from .env file
PG_HOST = os.getenv("PG_HOST")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_PORT = os.getenv("PG_PORT")

db_config = {
    "host": PG_HOST,
    "database": "international_week",
    "user": PG_USER,
    "password": PG_PASSWORD,
    "port": PG_PORT,
    "sslmode": "require"
}

soccer_anim = soccer_animation.SoccerAnimation(db_config)

animation_file = soccer_anim.animate_from_database(
    game_id='5uts2s7fl98clqz8uymaazehg',
    start_time='00:13:35',
    end_time='00:14:05',
    period_id=1,
    output_file='match_animation.mp4',
    fps=25,
    interpolate=True
)


Loading tracking data...
Frame range: 1722799715000 to 1722799744000
Loaded 690 rows, 30 unique frames
Loading team data...
Splitting tracking data...
Ball frames: 30
Home team rows: 330, players: 11
Away team rows: 330, players: 11
Creating animation with 30 original frames...
Interpolating ball frames...
Interpolating 30 frames to create 180 frames...


Interpolating positions: 100%|██████████| 29/29 [00:00<00:00, 176.81it/s]


After ball interpolation: 175 frames
Interpolating home team frames...
Interpolating 330 frames to create 1980 frames...
After home team interpolation: 1925 frames
Interpolating away team frames...
Interpolating 330 frames to create 1980 frames...
After away team interpolation: 1925 frames
Time range: 00:13:35 to 00:14:04


Processing frames:   1%|          | 1/175 [00:00<00:19,  9.11it/s]

Pre-processing frames...
Generating animation...
Saving animation to match_animation.mp4 with 25 fps...


Processing frames: 178it [00:09, 18.09it/s]                         

Animation completed!
Animation saved to match_animation.mp4
